In [37]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html5lib 
from lxml import html
import numpy as np
from helpers import *
#import helpers as helpers

In [29]:
# The URL of the "start" of the IS-Academia page listing the students.
home_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_x_GPS=-1&ww_i_reportModel=133685247"

# The general form of the URL, with the fields to be replaced later (e.g. [UNITE_ACADEMIQUE])
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=[UNITE_ACADEMIQUE]&ww_x_PERIODE_ACAD=[PERIODE_ACADEMIQUE]&ww_x_PERIODE_PEDAGO=[PERIODE_PEDAGOGIQUE]&ww_x_HIVERETE=null"

# Get the raw content from the page
with urllib.request.urlopen(home_url) as url:
    s = url.read()

soup = BeautifulSoup(s, 'html.parser')

# Parse the content
url_section = 0
url_years = {}
url_master = {}

#For loop on all the possible fields with <option> to get the value associated with each year, or each "Bachelor" semester
for link in soup.find_all('option'):
    if link.text == 'Informatique':
        url_section = link.get('value')
    if "20" in link.text:
        #print(link.text)
        url_years[link.text] = link.get('value')
    if 'Master' in link.text or 'Mineur' in link.text:
        url_master[link.text] = link.get('value')

        
print(url_years)
print(url_master)
#frame= frame.drop([2,3,4,5,6,8,9,11],axis=1)
#frame.head(10)

{'2010-2011': '39486325', '2014-2015': '213637922', '2013-2014': '213637754', '2016-2017': '355925344', '2007-2008': '978181', '2008-2009': '978187', '2009-2010': '978195', '2012-2013': '123456101', '2011-2012': '123455150', '2015-2016': '213638028'}
{'Projet Master printemps': '3781783', 'Master semestre 1': '2230106', 'Master semestre 3': '2230128', 'Projet Master automne': '249127', 'Master semestre 2': '942192', 'Mineur semestre 1': '2335667', 'Mineur semestre 2': '2335676', 'Master semestre 4': '2230140'}


In [44]:
def getFullUrl(PeriodeAcad, PeriodePedag):
    url = base_url
    url = url.replace('[UNITE_ACADEMIQUE]', str(url_section))
    url = url.replace('[PERIODE_ACADEMIQUE]', str(url_years[PeriodeAcad]))
    url = url.replace('[PERIODE_PEDAGOGIQUE]', str(url_master[PeriodePedag]))
    return url

Assumptions : 
- All students start in MS1 and do it only once since afterward it will be MS3.

In [114]:
master_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 'MS1', 'MS2', 'MS3', 'PDM Spring', 'PDM Autumn', 'Spécialisation', 'Mineur'])
master_student.index.name = 'No Sciper'

for year in range(2007,2016):
    year_string = str(year) + '-' + str(year+1)
    
    # MS1 : get all sciper of students in MS1 and add them all to the data frame with 1 in MS1 and 0 elsewhere
    current_url_MS1 = getFullUrl(year_string, 'Master semestre 1')
    with urllib.request.urlopen(current_url_MS1) as url:
        html_MS1 = url.read()
    soup = BeautifulSoup(html_MS1, 'html.parser')
    master_frame_MS1 = pd.read_html(soup.prettify(), header=1)
    master_frame_MS1 = master_frame_MS1[0].drop(0,axis=0)
    master_frame_MS1_sciper = master_frame_MS1[['No Sciper']]
    master_frame_MS1 = master_frame_MS1.set_index('No Sciper')
    for sciper in master_frame_MS1_sciper.values:
        sciper = sciper[0]
        #print(sciper)
        master_student.loc[sciper] = 0
        master_student.loc[sciper,'MS1'] = 1
        master_student.loc[sciper,'Civilité'] = master_frame_MS1.loc[sciper,'Civilité']
        master_student.loc[sciper,'Nom Prénom'] = master_frame_MS1.loc[sciper,'Nom Prénom']
        master_student.loc[sciper,'Spécialisation'] = master_frame_MS1.loc[sciper,'Spécialisation']
        master_student.loc[sciper,'Mineur'] = master_frame_MS1.loc[sciper,'Mineur']
    
    # MS2 : get all sciper of students in MS2 add 1 to MS2 field of students currently in frame 'master_student'
    current_url_MS2 = getFullUrl(year_string, 'Master semestre 2')
    with urllib.request.urlopen(current_url_MS2) as url:
        html_MS2 = url.read()
    soup = BeautifulSoup(html_MS2, 'html.parser')
    master_frame_MS2 = pd.read_html(soup.prettify(), header=1)
    master_frame_MS2 = master_frame_MS2[0].drop(0,axis=0)
    master_frame_MS2_sciper = master_frame_MS2[['No Sciper']]
    master_frame_MS2 = master_frame_MS2.set_index('No Sciper')
    for sciper in master_frame_MS2_sciper.values:
        sciper = sciper[0]
        if sciper in master_student.index:
            master_student.loc[sciper, 'MS2'] += 1
        # else ot means that student is in MS2 without having done MS1, or he has started before 2007 -> do not take
        # into account

    # Only keep students that are in their first 1st year
    #student_frame_B1 = student_frame_B1[[not (sciper in current_student.index) for sciper in student_frame_B1.index]]
    current_url_MS3 = getFullUrl(year_string, 'Master semestre 3')
    with urllib.request.urlopen(current_url_MS3) as url:
        html_MS3 = url.read()
    soup = BeautifulSoup(html_MS3, 'html.parser')
    master_frame_MS3 = pd.read_html(soup.prettify(), header=1)
    master_frame_MS3 = master_frame_MS3[0].drop(0,axis=0)
    master_frame_MS3_sciper = master_frame_MS3[['No Sciper']]
    master_frame_MS3 = master_frame_MS3.set_index('No Sciper')
    for sciper in master_frame_MS3_sciper.values:
        sciper = sciper[0]
        if sciper in master_student.index:
            master_student.loc[sciper, 'MS3'] += 1
    #current_student = pd.concat([current_student, student_frame_B1])

    
print(master_student.index.is_unique)
print(master_student.head(10))

True
           Civilité                 Nom Prénom  MS1  MS2  MS3  PDM Spring  \
No Sciper                                                                   
180027       Madame              Agarwal Megha  1.0  1.0  0.0         0.0   
152232     Monsieur         Anagnostaras David  1.0  1.0  1.0         0.0   
177395     Monsieur              Auroux Damien  1.0  1.0  1.0         0.0   
161970     Monsieur             Awalebo Joseph  1.0  1.0  1.0         0.0   
166258     Monsieur                  Balet Ken  1.0  1.0  1.0         0.0   
173600     Monsieur  Barazzutti Raphaël Pierre  1.0  1.0  0.0         0.0   
178879     Monsieur           Bayramoglu Ersoy  1.0  1.0  1.0         0.0   
154573       Madame         Benabdallah Zeineb  1.0  2.0  2.0         0.0   
160492     Monsieur                Bettex Marc  1.0  1.0  0.0         0.0   
172687     Monsieur               Billaud Joël  1.0  1.0  1.0         0.0   

           PDM Autumn Spécialisation Mineur  
No Sciper               

In [116]:
print(master_student.shape)
print(master_student.head(500))
#master_student.query('MS3==2')
master_student.describe()

(764, 9)
           Civilité                 Nom Prénom  MS1  MS2  MS3  PDM Spring  \
No Sciper                                                                   
180027       Madame              Agarwal Megha  1.0  1.0  0.0         0.0   
152232     Monsieur         Anagnostaras David  1.0  1.0  1.0         0.0   
177395     Monsieur              Auroux Damien  1.0  1.0  1.0         0.0   
161970     Monsieur             Awalebo Joseph  1.0  1.0  1.0         0.0   
166258     Monsieur                  Balet Ken  1.0  1.0  1.0         0.0   
173600     Monsieur  Barazzutti Raphaël Pierre  1.0  1.0  0.0         0.0   
178879     Monsieur           Bayramoglu Ersoy  1.0  1.0  1.0         0.0   
154573       Madame         Benabdallah Zeineb  1.0  2.0  2.0         0.0   
160492     Monsieur                Bettex Marc  1.0  1.0  0.0         0.0   
172687     Monsieur               Billaud Joël  1.0  1.0  1.0         0.0   
179802     Monsieur            Brightwell Mark  1.0  1.0  1.0      

,MS1,MS2,MS3,PDM Spring,PDM Autumn
count,764.0,764.000000,764.000000,764.0,764.0
mean,1.0,1.229058,0.657068,0.0,0.0
std,0.0,0.530727,0.588389,0.0,0.0
min,1.0,0.000000,0.000000,0.0,0.0
25%,1.0,1.000000,0.000000,0.0,0.0
50%,1.0,1.000000,1.000000,0.0,0.0
75%,1.0,2.000000,1.000000,0.0,0.0
max,1.0,3.000000,3.000000,0.0,0.0
